In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

__file__ = '/content/drive/MyDrive/aiffelthon/data'

Mounted at /content/drive


In [ ]:
from string import ascii_uppercase
#from draw_utils import *
#from pyglet.gl import *
import numpy as np
import pandas as pd
import os
import random
from datetime import datetime
import pytz
import matplotlib.pyplot as plt

###################################
# reward 조정
move_reward = -1  # 0.1
obs_reward = -10  # 0.1
goal_reward = 10  # 10
###################################
# train or test 모드 지정
train_mode = True
###################################
print('reward:' , move_reward, obs_reward, goal_reward)

#__file__ = '/home/ogangza/heung_path_finding/path-finding-rl/data'

local_path = os.path.abspath(os.path.join(os.path.dirname(__file__)))


class Simulator:
    def __init__(self):
        #height: 그리드 높이; width: 그리드 너비; inds: A ~ Q alphabet list
        #######################################################################################
        # Load train or test data
        if train_mode:  # 훈련 데이타 읽기
            self.files = pd.read_csv(os.path.join(local_path, "data/factory_order_train.csv"))
            print('data/factory_order_train.csv used')
        else:  # 테스트 데이터 읽기
            self.files = pd.read_csv(os.path.join(local_path, "data/factory_order_test.csv"))
            print('data/factory_order_test.csv used')
        #######################################################################################        
        self.height = 10
        self.width = 9
        self.inds = list(ascii_uppercase)[:17]

    def set_box(self):  # (수정사항) 목표물 그리드값 -100으로 지정하는 부분 코멘트 처리
        '''
        아이템들이 있을 위치를 미리 정해 놓고 그 위치 좌표들에 아이템이 들어올 수 있으므로 그리드에 100으로 표시한다.
        데이터 파일에서 이번 에피소드 아이템 정보를 받아 가져와야 할 아이템이 있는 좌표만 -100으로 표시한다.
        self.local_target에 에이전트가 이번에 방문해야할 좌표들을 저장한다.
        따라서 가져와야하는 아이템 좌표와 end point 좌표(처음 시작했던 좌표로 돌아와야 하므로)가 들어가게 된다.
        '''
        box_data = pd.read_csv(os.path.join(local_path, "./data/box.csv"))
        # 물건이 들어있을 수 있는 경우: 선반 위치. 그리드값 = 100
        for box in box_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(box, "row")][getattr(box, "col")] = 100
        # 물건이 실제 들어있는 경우: 정해진 목표. 그리드값 = -100
        order_item = list(set(self.inds) & set(self.items))
        order_csv = box_data[box_data['item'].isin(order_item)]
        for order_box in order_csv.itertuples(index = True, name ='Pandas'):
            ###########################################################################################################################
            self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = -100  # >>> main()에서 목표물 하나만 -100으로 바꾼다!!!
            ###########################################################################################################################
            # local_target에 가야 할 위치 좌표 넣기
            self.local_target.append([getattr(order_box, "row"), getattr(order_box, "col")])
        # self.local_target.sort() # 불필요. 인풋 데이터 A, B, C순 정렬되어 있음... 정렬 필요시 코드 바꾸어야 함
        ##################################################################
        self.local_target.append([9,4]) # 최종 목적지 (출발점) 추가
        ##################################################################
        # 알파벳을 Grid에 넣어서 -> grid에 2Dconv 적용 가능

    def set_obstacle(self):
        '''
        장애물이 있어야 하는 위치는 미리 obstacles.csv에 정의되어 있다. 이 좌표들을 0으로 표시한다.
        '''
        obstacles_data = pd.read_csv(os.path.join(local_path, "./data/obstacles.csv"))
        for obstacle in obstacles_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(obstacle, "row")][getattr(obstacle, "col")] = 0
        
    def reset(self, epi):
        '''
        reset()은 첫 스텝에서 사용되며 그리드에서 에이전트 위치가 start point에 있게 한다.
        :param epi: episode, 에피소드 마다 가져와야 할 아이템 리스트를 불러올 때 사용
        :return: 초기 셋팅된 그리드
        :rtype: numpy.ndarray
        _____________________________________________________________________________________
        items: 이번 에피소드에서 가져와야 하는 아이템들
        terminal_location: 현재 에이전트가 찾아가야 하는 목적지
        local_target: 한 에피소드에서 찾아가야하는 아이템 좌표, 마지막 엔드 포인트 등의 위치좌표들
        actions: visualization을 위해 에이전트 action을 저장하는 리스트
        curloc: 현재 위치
        '''
        # initial episode parameter setting
        self.epi = epi  # 에피소드 번호 받기
        self.items = list(self.files.iloc[self.epi])[0]  # 해당 에피소드의 items를 가져 옴. 예, [ 'H', 'L', 'M']
        self.cumulative_reward = 0
        #self.terminal_location = None
        self.local_target = []  # 목적지 리스트
        self.actions = []  # 지나 온 경로 리스트
        ###### initial grid setting
        self.grid = np.ones((self.height, self.width), dtype="float16")  # 그리드값 = 1... 10x9 크기의 그리드월드
        ###### set information about the gridworld
        self.set_box()  # 에이전트가 이번에 방문해야 할 좌표들 저장. 예, [ [0,3], [0,7], [0,8], [9,4] ]  # 그리드값 = 100(선반)
        self.set_obstacle()  # 그리드값 = 0
        ###### start point를 grid에 표시
        self.curloc = [9, 4]  # 출발점
        #self.grid[int(self.curloc[0])][int(self.curloc[1])] = -5  # 현재 위치(출발점) 그리드값 = -5
        self.done = False  # 종료 여부
        return self.grid

    def apply_action(self, action, cur_x, cur_y):
        '''
        에이전트가 행한 action대로 현 에이전트의 위치좌표를 바꾼다.
        action은 discrete하며 4가지 up,down,left,right으로 정의된다.
        :param x: 에이전트의 현재 x 좌표
        :param y: 에이전트의 현재 y 좌표
        :return: action에 따라 변한 에이전트의 x 좌표, y 좌표
        :rtype: int, int
        '''
        new_x = cur_x
        new_y = cur_y
        # up
        if action == 0:
            new_x = cur_x - 1
        # down
        elif action == 1:
            new_x = cur_x + 1
        # left
        elif action == 2:
            new_y = cur_y - 1
        # right
        else:
            new_y = cur_y + 1
        return int(new_x), int(new_y)

    def get_reward(self, new_x, new_y, out_of_boundary):  # 현재 목표에 도달한 경우... 코드 수정했음!!!
        '''
        get_reward함수는 리워드를 계산하는 함수이며, 상황에 따라 에이전트가 action을 옳게 했는지 판단하는 지표가 된다.
        :param new_x: action에 따른 에이전트 새로운 위치좌표 x
        :param new_y: action에 따른 에이전트 새로운 위치좌표 y
        :param out_of_boundary: 에이전트 위치가 그리드 밖이 되지 않도록 제한
        :return: action에 따른 리워드
        :rtype: float
        '''
        # 바깥으로 나가는 경우
        if any(out_of_boundary):
            reward = obs_reward   # -10점
        else:
            # 장애물에 부딪히는 경우 
            if self.grid[new_x][new_y] == 0:
                reward = obs_reward  # -10점
            #####################################################################
            # >>>>>>>>>> 현재 목표에 도달한 경우... 코드 수정했음!!!
            elif new_x == self.end_x and new_y == self.end_y:
                reward = goal_reward  # 10점
            #####################################################################
            # 그냥 움직이는 경우 
            else:
                reward = move_reward  # -1점
        return reward

    def step(self, action):  # 현재 목표에 도달한 경우... 수정 했음!!
        ''' 
        에이전트의 action에 따라 step을 진행한다.
        action에 따라 에이전트 위치를 변환하고, action에 대해 리워드를 받고, 어느 상황에 에피소드가 종료되어야 하는지 등을 판단한다.
        에이전트가 endpoint에 도착하면 gif로 에피소드에서 에이전트의 행동이 저장된다.
        :param action: 에이전트 행동
        :return:
            grid, 그리드
            reward, 리워드
            cumulative_reward, 누적 리워드
            done, 종료 여부
            goal_ob_reward, goal까지 아이템을 모두 가지고 돌아오는 finish율 계산을 위한 파라미터
        :rtype: numpy.ndarray, float, float, bool, bool/str
        (Hint : 시작 위치 (9,4)에서 up말고 다른 action은 전부 장애물이므로 action을 고정하는 것이 좋음)
        '''
        #self.terminal_location = self.local_target[0]  # (중간) 목적지 좌표 지정
        cur_x,cur_y = self.curloc  # 현재 위치 지정
        self.actions.append((cur_x, cur_y))  # 현재 위치를 (지나 온) 경로 리스트에 추가
        goal_ob_reward = False  # 0
        new_x, new_y = self.apply_action(action, cur_x, cur_y)  # 다음 위치 받기
        out_of_boundary = [new_x < 0, new_x >= self.height, new_y < 0, new_y >= self.width]  # 밖으로 나가면 OB = True 판정
        # 바깥으로 나가는 경우 종료
        if any(out_of_boundary):
            #print('OB')
            #원 위치
            #new_x = cur_x
            #new_y = cur_y
            self.done = True  # 종료
            self.grid[cur_x][cur_y] = 1   # 현재의 그리드값 = 1 (초기값)로 바꿈
            #goal_ob_reward = True  # 1
        else:
            # 장애물에 부딪히는 경우 종료
            if self.grid[new_x][new_y] == 0:
                #print('장애물')
                #원 위치
                #new_x = cur_x
                #new_y = cur_y
                self.done = True  # 종료
                self.grid[cur_x][cur_y] = 1   # 현재의 그리드값 = 1 (초기값)로 바꿈
                #goal_ob_reward = True  # 1
            #################################################################################
            # 현재 목표점에 도착한 경우 종료
            #################################################################################
            elif new_x == self.end_x and new_y == self.end_y:
                #print('목적지 도착')
                self.done = True  # 종료
                #self.local_target.remove(self.local_target[0])  # 다음 목표 설정을 위해 달성한 목표 제거
                #self.grid[cur_x][cur_y] = 1  # 현재의 그리드값 = 1로 바꿈
                #self.grid[new_x][new_y] = -5  # 이동한 그리드값 = -5로 바꿈
                goal_ob_reward = True  # 1
                #self.curloc = [new_x, new_y]  # 현재 위치를 이동한 그리드 위치로 변경
            else:
                # 그냥 움직이는 경우
                #print('이동')
                self.grid[cur_x][cur_y] = 1   # 현재의 그리드값 = 1 (초기값)로 바꿈
                self.grid[new_x][new_y] = -5  # 이동한 그리드값 = -5 (현 위치)로 바꿈
                self.curloc = [new_x,new_y]

        reward = self.get_reward(new_x, new_y, out_of_boundary)
        self.cumulative_reward += reward
        return self.grid, reward, self.cumulative_reward, self.done, goal_ob_reward


reward: -1 -10 10


In [ ]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Hyperparameters
learning_rate = 1e-3  # 0.0005
gamma         = 0.90  # 0.98
buffer_limit  = 50000
batch_size    = 32

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        #return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               #torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               #torch.tensor(done_mask_lst)
        ##############################################################################
        ## *_list를 np.array(*_lst)로 변경
        ##############################################################################
        return torch.tensor(np.array(s_lst), dtype=torch.float), torch.tensor(np.array(a_lst)), \
               torch.tensor(np.array(r_lst)), torch.tensor(np.array(s_prime_lst), dtype=torch.float), \
               torch.tensor(np.array(done_mask_lst))
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()

        in_size = 90  # input 크기
        L1 = 128  
        L2 = 512
        out_size  = 4

        self.fc1 = nn.Linear(in_size, L1)
        self.fc2 = nn.Linear(L1, L2)
        self.fc3 = nn.Linear(L2, out_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,3)
        else : 
            return out.argmax().item()
            
def train(q, q_target, memory, optimizer):  # 경로 하나가 완료되면 실행
    for i in range(10):  # 미니배치를 10번 뽑아서 그래디언트 업데이트
        s,a,r,s_prime,done_mask = memory.sample(batch_size)  # 리플레이 버퍼에서 미니배치를 뽑고... batch_size=32

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss.data

In [ ]:

#def main():
tz = pytz.timezone('Asia/Seoul')
    ####################################################################################
    #for n_epi in range(1000): # range()의 인수로 len(env.files) 사용하면 됨 (=39,999)
    ####################################################################################
env = Simulator()
q = Qnet()
q_target = Qnet()
q_target.load_state_dict(q.state_dict())
memory = ReplayBuffer()
optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    #files = pd.read_csv("./data/factory_order_train.csv") # Simulator 클래스 __init__(self):에서 읽음
print('len(env.files):', len(env.files))


# >>>>>>>>> 전체 훈련 데이터에 대한 최적 경로 찾기 훈련 시작 <<<<<<<<<<<<<<<<<<<<<
for n_epi in range(1000): # range()의 인수로 len(env.files) 사용하면 됨 (=39,999)  # <<<<<<<<<<<<<<<<<<<< 일단 1000개 경로 훈련!!!!

    env.grid = env.reset(n_epi)  # env reset: 에피소드 번호에 해당하는 목표물 리스트 env.local_target 생성, 그리드맵 생성
    print('▶ Episode #', n_epi,':', list(env.files.iloc[n_epi])[0])
    #print('list(env.files.iloc[',0,'])[0]:', list(env.files.iloc[0])[0])
    print('env.local_target:', env.local_target)
    #print("reset(0) 결과로 받은 최초 그리드맵")
    #print(env.grid)

    # >>>>>>>>> 하나의 에피소드에 대한 대한 최적 경로 찾기 시작 <<<<<<<<<<<<<<<<<<<<<
    for n_target in range(len(env.local_target)):
        sync_freq = 500  # q 네트워크 파라미터 q_target 네트워크 복사 주기 <<<<<<<<<<<<<
        path_length = 0  # 경로 길이 초기화
        # >>>>>>>>>>>>>> 출발지, 목적지 지정
        if n_target == 0:
            start_point = [9,4]  # 첫 번째 출발지 지정
            print('출발지:', start_point, end=" → ")
        else:
            start_point = env.local_target[n_target-1]
            print('출발지:', start_point, end=" → ")
        end_point = env.local_target[n_target]  # 목적지 지정
        print('목적지:', end_point)
        loss = None
        cnt = 0  # for loop 조기 종료를 위한 카운터

        # >>>>>>>>>>>>>>>>>>>>> 하나의 목적지에 대한 최적 경로 찾기 시작 <<<<<<<<<<<<<<<<<<<<<
        for iter in range(5000): # 최적 반복 횟수 결정 필요. 카트폴: 10000, 논문: 5000  <<<<<<<<<<<<<<
            epsilon = max(0.01, 0.08 - 0.01*(iter/200)) #Linear annealing from 8% to 1%
            # 출발지, 목적지 좌표 지정
            env.x, env.y = start_point                         # 출발지 좌표
            env.grid[int(env.x)][int(env.y)] = -5              # 출발지 그리드값 = -5
            env.end_x, env.end_y = end_point                   # 목적지 좌표
            env.grid[int(env.end_x)][int(env.end_y)] = -200    # 목적지 그리드값 = -200
            # 현 위치를 출발지로 reset
            env.curloc = start_point
            # 목적지 도착 flag (done), 인풋 데이터 (s) 등 초기화
            env.done = False
            done = False
            env.actions = []
            #######################################################################
            ## 인풋 정보: 그리드맵... 전체 1로 초기화 후, 선반 100, 장애물 0, 목적지 리스트: -100, 이번 목적지 -200, 현 위치 -5
            #######################################################################
            grid_map = env.grid.reshape(-1)          # 그리드 맵
            #print('현 위치(-5), 목적지(-200) 표시 그리드맵: >>> 시작')
            #print(grid_map.reshape(10,9))
            #######################################################################
            s = np.array(grid_map)
            while not done:  # 한 칸씩 이동. 목적지 도착하면 종료
                #print('>>>>>>>>>>>> while 시작점... current location:', env.curloc)
                #####################################################################################
                # (option) 출발점에서는 무조건 위로 올라간다 (action=0)
                if env.curloc == [9,4]:
                    a = 0
                else:
                    a = q.sample_action(torch.from_numpy(s).float(), epsilon)  # e-greedy로 액션 선택... 네트워크 결과값으로
                #####################################################################################
                #a = q.sample_action(torch.from_numpy(s).float(), epsilon)  # e-greedy로 액션 선택... 네트워크 결과값으로
                grid, r, cum_reward, done, goal_ob_reward = env.step(a)
                new_grid_map = grid.reshape(-1)
                #print('액션=', a, '한 칸 이동 결과', 'done=', done)
                #print(new_grid_map.reshape(10,9))
                s_prime = np.array(new_grid_map)
                done_mask = 0.0 if done else 1.0
                memory.put((s,a,r/100.0,s_prime, done_mask))
                s = s_prime
                if done:
                    break  #  while loop 빠져 나가기 
                # while loop 종료
            path_length += len(env.actions)
            ############################################################################
            ## memory.size(): 2000... 이후 훈련 시작
            ############################################################################   
            if memory.size() > 2000:
                loss = train(q, q_target, memory, optimizer)
                ## for loop 종료 조건 만들어 넣기 필요 <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                if loss < 1e-3:
                    cnt += 1
                else:
                    cnt = 0
                if cnt > 10:  # 10회 연속 loss값이 0.001 보다 작으면
                    print('10회 연속 loss값이 0.001 보다 작음 @ iter#', iter)
                    break  #  for loop 빠져 나가기
            if iter % sync_freq == 0 and iter != 0:
                #cur_time = datetime.now(tz)
                #simple_cur_time = cur_time.strftime("%H:%M:%S")
                #print('▶ Episode #', iter, 'start time:', simple_cur_time, end='→')
                q_target.load_state_dict(q.state_dict())  # 얼마나 자주 q 네트워크 파라미터를 q_target 네트워크로 복사할 것인가???
                print("iteration:{}, 평균 보상: {:.1f}, 버퍼크기: {}, 엡실론: {:.1f}%".\
                      format(iter, env.cumulative_reward/sync_freq, memory.size(), epsilon*100))
                print('loss:', loss, '→ 평균 경로 길이:', path_length/sync_freq)
                path_length = 0.0
                env.cumulative_reward = 0.0
            #torch.save(q, '/content/drive/MyDrive/aiffelthon/data/model.pt')
        print('for loop 종료')
        # >>>>>>>>>>>>>>>>>>>>> 하나의 목적지에 대한 최적 경로 찾기 종료 <<<<<<<<<<<<<<<<<<<<<
    # >>>>>>>>> 하나의 에피소드에 대한 대한 최적 경로 찾기 종료 <<<<<<<<<<<<<<<<<<<<<
# >>>>>>>>> 전체 훈련 데이터에 대한 최적 경로 찾기 훈련 종료 <<<<<<<<<<<<<<<<<<<<<

In [ ]:
#train 모드 실행
#main()

In [ ]:
## test 메서드 추가 #################################################################################
def test():
    model = torch.load('/content/drive/MyDrive/aiffelthon/data/model.pt')
    train_mode = False  # False
    tz = pytz.timezone('Asia/Seoul')
    env = Simulator()
    q = Qnet()
    s_before = None
    #sync_freq = 1  # 
    score = 0.0

    for n_epi in range(1): # range()의 인수로 len(env.files) 사용하면 됨 (=1225)
        env.reset(n_epi)
        print('env.local_target:', env.local_target)
        ############################################
        ## 인풋 정보(obs)에 경로 길이 추가
        #path_length = len(env.local_target)
        #print('path_length', path_length)
        #s_before = env.curloc
        #s_before.append(path_length)
        ############################################        
        s = np.array(s_before)
        done = False

        while not done:  # OB, 장애물, 최종 목표 도달시 종료
            #####################################################################################
            # (option) test 경우 추가: 출발점에서는 무조건 위로 올라간다 (action=0)
            x, y, _ = s
            if [x,y] == [9,4]:
                a = 0
            else:
                a = q.test_action(torch.from_numpy(s).float())  # test_action 수행
            #####################################################################################
            #a = q.test_action(torch.from_numpy(s).float())  # test_action 수행
            #####################################################################################
            obs, r, cum_reward, done, goal_ob_reward = env.step(a)
            s_prime = np.array(env.curloc)
            s = s_prime
            score += r

        # sync_freq (=1) 마다 현황 디스플레이하고 q_target 업데이트 및 score 초기화
        if n_epi % sync_freq == 0 and n_epi != 0:
            cur_time = datetime.now(tz)
            simple_cur_time = cur_time.strftime("%H:%M:%S")
            print('▶ Episode #', n_epi, end=' → ')
            print('Score =', score, end='...')
            print('경로 길이:', len(env.actions), end=', ')
            if goal_ob_reward == 'finish':
                print('성공 여부: ', goal_ob_reward)
            else:
                print('성공 여부 : 실패', goal_ob_reward)
        
        score = 0.0
        print('→ pred_path:', env.actions)

    # 모든 에피소드 종료 후 결과 디스플레이
    # 코드 추가할 것!!!
'''